In [43]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
import seaborn as sns
from tensorflow.keras import regularizers
from sklearn.metrics import classification_report,confusion_matrix
from tensorflow.keras import layers
from tensorflow.keras import losses

from collections import Counter
from sklearn.metrics import (precision_score, recall_score,
                             f1_score, accuracy_score,mean_squared_error,mean_absolute_error)

import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from tensorflow.keras import preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.datasets import imdb
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.naive_bayes import GaussianNB


In [44]:
#uploading training dataset
test_data= pd.read_csv("UNSW_NB15_testing-set.csv",header=None)
print('-------Test data--------')
print(test_data[43].value_counts())
print(test_data.shape)
print('-------------------------')
test_data.drop(test_data.index[0],inplace = True)
test_data

-------Test data--------
Normal            56000
Generic           40000
Exploits          33393
Fuzzers           18184
DoS               12264
Reconnaissance    10491
Analysis           2000
Backdoor           1746
Shellcode          1133
Worms               130
Name: 43, dtype: int64
(175341, 45)
-------------------------


,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,40,41,42,43,44
1,2,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,...,1,2,0,0,0,1,6,0,Normal,0
2,3,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,...,1,3,0,0,0,2,6,0,Normal,0
3,4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,...,1,3,1,1,0,2,1,0,Normal,0
4,5,0.449454,tcp,-,FIN,10,6,534,268,33.373826,...,1,40,0,0,0,2,39,0,Normal,0
5,6,0.380537,tcp,-,FIN,10,6,534,268,39.417980,...,1,40,0,0,0,2,39,0,Normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175336,175337,0.000009,udp,dns,INT,2,0,114,0,111111.107200,...,13,24,0,0,0,24,24,0,Generic,1
175337,175338,0.505762,tcp,-,FIN,10,8,620,354,33.612649,...,1,2,0,0,0,1,1,0,Shellcode,1
175338,175339,0.000009,udp,dns,INT,2,0,114,0,111111.107200,...,3,13,0,0,0,3,12,0,Generic,1
175339,175340,0.000009,udp,dns,INT,2,0,114,0,111111.107200,...,14,30,0,0,0,30,30,0,Generic,1


In [45]:
test_data.drop(test_data[test_data[43]=='Generic'].index, inplace = True)
test_data.drop(test_data[test_data[43]=='Fuzzers'].index, inplace = True)
test_data.drop(test_data[test_data[43]=='Reconnaissance'].index, inplace = True)
test_data.drop(test_data[test_data[43]=='Analysis'].index, inplace = True)
test_data.drop(test_data[test_data[43]=='Backdoor'].index, inplace = True)
test_data.drop(test_data[test_data[43]=='Shellcode'].index, inplace = True)
test_data.drop(test_data[test_data[43]=='Worms'].index, inplace = True)
print(test_data[43].value_counts())

Normal      55999
Exploits    33393
DoS         12264
Name: 43, dtype: int64


In [46]:
train_data= pd.read_csv("UNSW_NB15_training-set.csv",header= None)
print('-------Train data--------')
print(train_data[43].value_counts())
print(len(train_data))
print('-------------------------')
train_data.drop(train_data.index[0],inplace = True)
train_data

C:\Users\DELL\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0,1,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,44) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


-------Train data--------
Normal            37000
Generic           18871
Exploits          11132
Fuzzers            6062
DoS                4089
Reconnaissance     3496
Analysis            677
Backdoor            583
Shellcode           378
Worms                44
attack_cat            1
Name: 43, dtype: int64
82333
-------------------------


,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,40,41,42,43,44
1,1,0.000011,udp,-,INT,2,0,496,0,90909.0902,...,1,2,0,0,0,1,2,0,Normal,0
2,2,0.000008,udp,-,INT,2,0,1762,0,125000.0003,...,1,2,0,0,0,1,2,0,Normal,0
3,3,0.000005,udp,-,INT,2,0,1068,0,200000.0051,...,1,3,0,0,0,1,3,0,Normal,0
4,4,0.000006,udp,-,INT,2,0,900,0,166666.6608,...,1,3,0,0,0,2,3,0,Normal,0
5,5,0.00001,udp,-,INT,2,0,2126,0,100000.0025,...,1,3,0,0,0,2,3,0,Normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82328,82328,5e-06,udp,-,INT,2,0,104,0,200000,...,1,2,0,0,0,2,1,0,Normal,0
82329,82329,1.1061,tcp,-,FIN,20,8,18062,354,24.4101,...,1,1,0,0,0,3,2,0,Normal,0
82330,82330,0,arp,-,INT,1,0,46,0,0,...,1,1,0,0,0,1,1,1,Normal,0
82331,82331,0,arp,-,INT,1,0,46,0,0,...,1,1,0,0,0,1,1,1,Normal,0


In [47]:
train_data.drop(train_data[train_data[43]=='Generic'].index, inplace = True)
train_data.drop(train_data[train_data[43]=='Fuzzers'].index, inplace = True)
train_data.drop(train_data[train_data[43]=='Reconnaissance'].index, inplace = True)
train_data.drop(train_data[train_data[43]=='Backdoor'].index, inplace = True)
train_data.drop(train_data[train_data[43]=='Analysis'].index, inplace = True)
train_data.drop(train_data[train_data[43]=='Shellcode'].index, inplace = True)
train_data.drop(train_data[train_data[43]=='Worms'].index, inplace = True)
print(train_data[43].value_counts())

Normal      37000
Exploits    11132
DoS          4089
Name: 43, dtype: int64


In [48]:
train_data.isnull().sum()
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52221 entries, 1 to 82332
Data columns (total 45 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       52221 non-null  object
 1   1       52221 non-null  object
 2   2       52221 non-null  object
 3   3       52221 non-null  object
 4   4       52221 non-null  object
 5   5       52221 non-null  object
 6   6       52221 non-null  object
 7   7       52221 non-null  object
 8   8       52221 non-null  object
 9   9       52221 non-null  object
 10  10      52221 non-null  object
 11  11      52221 non-null  object
 12  12      52221 non-null  object
 13  13      52221 non-null  object
 14  14      52221 non-null  object
 15  15      52221 non-null  object
 16  16      52221 non-null  object
 17  17      52221 non-null  object
 18  18      52221 non-null  object
 19  19      52221 non-null  object
 20  20      52221 non-null  object
 21  21      52221 non-null  object
 22  22      52221 non-null

In [49]:
train_data=train_data.drop([2,3,43],axis=1)
train_data=train_data.drop([0],axis=1)
test_data=test_data.drop([2,3,43],axis=1)
test_data=test_data.drop([0],axis=1)

In [50]:
from sklearn import preprocessing
le=preprocessing.LabelEncoder()
'''train_data[43] = le.fit_transform(train_data[43])
test_data[43] = le.fit_transform(test_data[43])'''
train_data[4] = le.fit_transform(train_data[4])
test_data[4] = le.fit_transform(test_data[4])

In [51]:
train_data = train_data.apply(pd.to_numeric)
test_data = test_data.apply(pd.to_numeric)

In [52]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52221 entries, 1 to 82332
Data columns (total 41 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   1       52221 non-null  float64
 1   4       52221 non-null  int32  
 2   5       52221 non-null  int64  
 3   6       52221 non-null  int64  
 4   7       52221 non-null  int64  
 5   8       52221 non-null  int64  
 6   9       52221 non-null  float64
 7   10      52221 non-null  int64  
 8   11      52221 non-null  int64  
 9   12      52221 non-null  float64
 10  13      52221 non-null  float64
 11  14      52221 non-null  int64  
 12  15      52221 non-null  int64  
 13  16      52221 non-null  float64
 14  17      52221 non-null  float64
 15  18      52221 non-null  float64
 16  19      52221 non-null  float64
 17  20      52221 non-null  int64  
 18  21      52221 non-null  int64  
 19  22      52221 non-null  int64  
 20  23      52221 non-null  int64  
 21  24      52221 non-null  float64
 22

In [53]:
train_data.round(decimals=2)
train_data = train_data.astype(int)
test_data.round(decimals=2)
test_data = test_data.astype(int)
train_data

,1,4,5,6,7,8,9,10,11,12,...,34,35,36,37,38,39,40,41,42,44
1,0,3,2,0,496,0,90909,254,0,180363632,...,1,1,2,0,0,0,1,2,0,0
2,0,3,2,0,1762,0,125000,254,0,881000000,...,1,1,2,0,0,0,1,2,0,0
3,0,3,2,0,1068,0,200000,254,0,854400000,...,1,1,3,0,0,0,1,3,0,0
4,0,3,2,0,900,0,166666,254,0,600000000,...,2,1,3,0,0,0,2,3,0,0
5,0,3,2,0,2126,0,100000,254,0,850400000,...,2,1,3,0,0,0,2,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82328,0,3,2,0,104,0,200000,254,0,83200000,...,1,1,2,0,0,0,2,1,0,0
82329,1,2,20,8,18062,354,24,254,252,124104,...,1,1,1,0,0,0,3,2,0,0
82330,0,3,1,0,46,0,0,0,0,0,...,1,1,1,0,0,0,1,1,1,0
82331,0,3,1,0,46,0,0,0,0,0,...,1,1,1,0,0,0,1,1,1,0


In [54]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52221 entries, 1 to 82332
Data columns (total 41 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   1       52221 non-null  int32
 1   4       52221 non-null  int32
 2   5       52221 non-null  int32
 3   6       52221 non-null  int32
 4   7       52221 non-null  int32
 5   8       52221 non-null  int32
 6   9       52221 non-null  int32
 7   10      52221 non-null  int32
 8   11      52221 non-null  int32
 9   12      52221 non-null  int32
 10  13      52221 non-null  int32
 11  14      52221 non-null  int32
 12  15      52221 non-null  int32
 13  16      52221 non-null  int32
 14  17      52221 non-null  int32
 15  18      52221 non-null  int32
 16  19      52221 non-null  int32
 17  20      52221 non-null  int32
 18  21      52221 non-null  int32
 19  22      52221 non-null  int32
 20  23      52221 non-null  int32
 21  24      52221 non-null  int32
 22  25      52221 non-null  int32
 23  26      522

In [55]:
features = train_data.iloc[:,0:40]
target = train_data.iloc[:,40]
X_Train,X_Test,Y_Train,Y_Test=train_test_split(features,target,test_size = 0.2,random_state=2)

In [61]:
model= GaussianNB()
model.fit(X_Train, Y_Train)
prediction= model.predict(X_Test) #prediction based on testing_set
accuracy = accuracy_score(Y_Test, prediction)
recall = recall_score(Y_Test, prediction , average="weighted")
precision = precision_score(Y_Test, prediction , average="weighted")
f1 = f1_score(Y_Test, prediction, average="weighted")
print("confusion matrix")
print("----------------------------------------------")
print("accuracy")
print("%.3f" %accuracy)
print("racall")
print("%.3f" %recall)
print("precision")
print("%.3f" %precision)
print("f1score")
print("%.3f" %f1)
expected= Y_Train #expected result stored in training_set
predicted= model.predict(X_Train)
print('Classification report:')
print(classification_report(expected,predicted)) #calculating classification report
print('Confusion matrix:')
print(confusion_matrix(expected,predicted)) #calculating confusion matrix


confusion matrix
----------------------------------------------
accuracy
0.720
racall
0.720
precision
0.686
f1score
0.666
Classification report:
              precision    recall  f1-score   support

           0       0.74      0.95      0.83     29615
           1       0.58      0.18      0.27     12161

    accuracy                           0.72     41776
   macro avg       0.66      0.56      0.55     41776
weighted avg       0.69      0.72      0.67     41776

Confusion matrix:
[[28067  1548]
 [10007  2154]]
